In [6]:
#from  scapy.all import *
import nest_asyncio
nest_asyncio.apply()
import pandas as pd
from scapy.all import *
#files name in pcap direcoty 
import glob

In [7]:
#Take all split values from folder, that I just done above

In [8]:
name = 'logistic'

In [9]:
#split_files = glob.glob("NF_exponential/*.pcap")
split_files = glob.glob("NF_"+name+"/*.pcap")

In [10]:
split_files

['NF_logistic/1_host_5_logistic_2.pcap',
 'NF_logistic/1_host_5_logistic_1.pcap']

In [11]:
total_split_files = len(split_files)

In [12]:
total_split_files

2

In [13]:
names = split_files[0].split('.')


In [14]:
node_number = names[0].split('/')

In [15]:
node_number[1]

'1_host_5_logistic_2'

In [17]:
for s_files in range(total_split_files):
    print(split_files[s_files])
    pcap =rdpcap(split_files[s_files])
    reset= []
    for p in pcap:
        if p.haslayer(TCP) and (p[TCP].flags == 'RA' or p[TCP].flags == 'R') : #reset,ack or reset flag RA mean both reset and ack
            if p[IP].src == '192.168.0.150': #server port
                #print("2015-----")
                #print(p[TCP].sport)
                #print(p[TCP].dport)
                reset.append(p[TCP].dport)
            else:
                #print("not 2015---")
                reset.append(p[TCP].sport)
                #print(p[TCP].sport)
                #print(p[TCP].dport)
            
    #head= []

    #for p in pcap:
        #if p.haslayer(TCP): 
            #if Raw in p[TCP]:
                #raw = str(p[Raw])
                #if 'HEAD' in raw:
                    #head.append(p[TCP].sport)   

#print(head)


    connection=[]; connection_sip=[]; connection_dip=[]; connection_sport=[]; connection_sip=[]
    c=0
    #pcap[1].show()  
    extra=0; segg=1; array=[]; array2=[]
    for ppp in pcap:
        # Select full connection except reset and head connection 
        if (ppp.haslayer(TCP)) and ( ppp[TCP].sport not in reset):
            #print('fuu'+str(ppp[TCP].sport))
            if Raw in ppp[TCP]:
                if(ppp[IP].dst == '192.168.0.150'):# server ip
                    if 'GET' in str(ppp[Raw]):
                        connection.append(c)
                        c=c+1
                        connection_sip.append(ppp[IP].src)
                        connection_sport.append(ppp[TCP].sport)
                        if ("sintel_enc_x264_dash.mpd" in str(ppp[Raw]) or "sintel_enc_x264_dash_init.mp4"  in str(ppp[Raw])):
                                extra=extra+1
                        #elif 'segment'+str(segg) in str(ppp[Raw]):
                         #           array.append(ppp[TCP].sport)
                          #          segg=segg+1
                        #else:
                            #array2.append(segg)
                            #segg=segg+1
                            #continue
                        #print(array)
                        #print(segg)
    #print(extra)                  
    conn=[] ;ij=extra-1;ik=0            
    conn=sorted(list(set(connection_sport)))                    
    for q in range(len(conn)-extra):
         for qq in pcap:
            if  qq.haslayer(TCP) and ((qq[TCP].sport == conn[q+extra])):
                 if Raw in qq[TCP]:
                    if(qq[IP].dst == '192.168.0.150'):# server ip
                        if 'GET' in str(qq[Raw]):
                                #print(str((q+extra)-ij) )
                                if ((q+extra)-ij)>59:
                                    break
                                if 'segment'+str((q+extra)-ij) in str(qq[Raw]):
                                    array.append(qq[TCP].sport)
                                    #print(str((q+extra)-ij) )
                                    #print("conn:"+str(conn[(q+extra)-ij]))
                                    #print(array)
                                    segg=segg+1
                                    #ij=3
                                #print(segg)
                                #perint
                                else:
                                    ik=ik+1
                                    array2.append((q+extra)-ij)
                                    ij=extra-(1+ik)
    df= pd.DataFrame(columns=['Port','Segment','RTT','Throughput', 'Packets'])# rtt=uplink ms, th=downlink byte per sec, sp= downlink packets

    t1=0;t2=0;l=0;inner =True;k=0;b=True;s=0;

    i=1; 
    length= (len(connection))-extra
    #print(extra)
    #print(length)
    m=extra
    for j in range(len(conn)-m):
        pac1= True;pac= True; ttt=True;f=True; c=0;tc= True; r1=0; initrtt=0; r2=0;  inner=True; inner2=False;httprtt=0; aaa=0;summ=0;l=0;tspop=0
        for p in pcap:
            if p.haslayer(TCP) and (p[TCP].sport == conn[j+m] or p[TCP].dport== conn[j+m]):
                if pac:
                    p1_t=p.time
                    pac=False
                if(p[IP].dst == connection_sip[1]):  
                    if(  p[TCP].flags == 'SA'):
                        if(tc==True):
                            p2_t= p.time
                            initrtt= (p2_t - p1_t) * 1000
                            #print("initrtt" + str(initrtt))
                            tc=False

                if  p[IP].src == connection_sip[j+m] and pac1 and  Raw in p[TCP]:
                                                    #if (segment in str(p[Raw])):
                    r1 = p.time
                    #print('r1---'+ str(r1))
                    pac1 = False
                    #print(p[IP].src)

                if p[IP].dst == connection_sip[j+m]:

                        if Raw in p[TCP]:
                            l=l+int(len(p[Raw]))
                            if len(p[TCP]) > 32:
                                    pcakets=p[TCP]
                                    tstop=p.time
                                    c=c+1  

                        if ttt and (p[TCP].flags == 'A' or p[TCP].flags == 'PA'):
                            r2=p.time
                            httprtt= r2 - r1
                            #print('r2---'+ str(r2))
                            aaa= httprtt * 1000
                            summ= (initrtt+aaa)/2
                            ttt=False    

                if p[TCP].flags == 'FA':# 
                    th=float(l/(tstop-r1))
                    if j+m in array2:
                        i=i+1
                        continue


                    #df.loc[i]=[aaa ,th ,c ]
                    df = df.append(pd.Series([conn[j+m], i, aaa ,th ,c], index=df.columns ), ignore_index=True)
                    i=i+1
                    #print(df)

                    break
    names = split_files[s_files].split('.')

    node_number = names[0].split('/')
        
    
    export_csv = df.to_csv ("NF_"+name+"/qos_"+name+"/"+node_number[1]+".csv", index = None, header=True) #Don't forget   
    print('Done') 

NF_logistic/1_host_5_logistic_2.pcap
Done
NF_logistic/1_host_5_logistic_1.pcap
Done
